In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pyplot

In [2]:
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure

In [3]:
# loading data from parquet file
import os
file_path = os.path.join(os.getcwd(), 'yellow_tripdata_2024-05.parquet')

df = pd.read_parquet(file_path)


In [4]:
df.sample(10)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
2499708,1,2024-05-23 13:46:42,2024-05-23 14:05:20,1.0,1.80,1.0,N,143,162,4,17.00,2.5,0.5,0.00,0.00,1.0,21.00,2.5,0.0
3380126,2,2024-05-05 16:17:52,2024-05-05 16:23:47,NaN,1.24,NaN,NaN,236,236,0,13.78,0.0,0.5,0.00,0.00,1.0,17.78,NaN,NaN
1361050,1,2024-05-13 09:45:41,2024-05-13 10:12:27,1.0,2.30,1.0,N,236,100,1,21.20,2.5,0.5,5.00,0.00,1.0,30.20,2.5,0.0
972787,2,2024-05-09 18:07:08,2024-05-09 18:38:23,1.0,3.22,1.0,N,233,238,1,27.50,2.5,0.5,6.80,0.00,1.0,40.80,2.5,0.0
268669,2,2024-05-03 11:52:08,2024-05-03 12:38:08,1.0,13.73,1.0,N,233,123,1,62.50,0.0,0.5,14.69,6.94,1.0,88.13,2.5,0.0
2628963,2,2024-05-24 16:09:54,2024-05-24 17:08:18,6.0,4.53,1.0,N,236,211,1,47.80,2.5,0.5,30.00,0.00,1.0,84.30,2.5,0.0
1507268,2,2024-05-14 16:03:33,2024-05-14 16:07:02,2.0,0.53,1.0,N,236,263,1,5.80,0.0,0.5,1.50,0.00,1.0,11.30,2.5,0.0
309811,2,2024-05-03 18:52:57,2024-05-03 19:01:11,2.0,1.83,1.0,N,162,236,1,10.00,2.5,0.5,1.98,0.00,1.0,18.48,2.5,0.0
3229095,2,2024-05-31 09:29:52,2024-05-31 09:44:57,1.0,1.84,1.0,N,170,50,1,14.90,0.0,0.5,0.00,0.00,1.0,18.90,2.5,0.0
2031065,1,2024-05-18 22:45:09,2024-05-18 22:46:55,1.0,0.00,5.0,N,151,151,1,30.00,0.0,0.0,0.01,0.00,1.0,31.01,0.0,0.0


In [5]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 3723833 entries, 0 to 3723832
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     str           
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee            float64   

# Checking the null values 

In [6]:
na_count = df.isnull().sum()
na_pct = (
    df
    .isnull()
    .sum()
    .div(df.shape[0])
    .mul(100)
    .round(2)
    
)

(
    pd
    .concat([na_count, na_pct], axis=1)
    .set_axis(['Count', '%'], axis=1)
    .query('Count > 0')
    .sort_values('Count', ascending=False)
)

,Count,%
passenger_count,404666,10.87
RatecodeID,404666,10.87
store_and_fwd_flag,404666,10.87
congestion_surcharge,404666,10.87
Airport_fee,404666,10.87


## found around 10% of the data has null values

# Checking duplicate entires 

In [38]:
df.duplicated().sum()

np.int64(0)

# Now will be working with mongodb using pymongo

## Estabishing connection with mongo db

In [7]:
try:
    client = MongoClient("mongodb://localhost:27017/")

    db = client["birajit"]
    collection = db["taxidata"]

except Exception as e:
    print(e)



In [11]:
CHUNK_SIZE = 25_000

for start in range(0, len(df), CHUNK_SIZE):
    chunk = df.iloc[start:start + CHUNK_SIZE]
    collection.insert_many(chunk.to_dict("records"), ordered=False)